In [1]:
import pandas as pd
import numpy as np
import re
import requests
from bs4 import BeautifulSoup

In [2]:
tw = pd.read_csv('/Users/luismoreira/Desktop/twitter_project/stock_tweets/tweets_2010_2020/2020.csv')

In [3]:
tw = tw.drop(['Unnamed: 0', 'id','conversation_id', 'created_at', 'timezone', 'place','language', 'hashtags', 'cashtags', 'user_id', 'user_id_str',
       'username', 'name', 'day', 'hour', 'link', 'urls', 'photos', 'video',
       'thumbnail', 'retweet', 'nlikes', 'nreplies', 'nretweets', 'quote_url',
       'search', 'near', 'geo', 'source', 'user_rt_id', 'user_rt',
       'retweet_id', 'reply_to', 'retweet_date', 'translate', 'trans_src',
       'trans_dest'], axis = 1)

In [4]:
tw

,date,tweet
0,2020-12-28 19:46:18,Entertainment will be critical when cars drive...
1,2020-12-28 18:07:33,@kimpaquette Just meeting with Larry Ellison t...
2,2020-12-28 17:23:51,@richierichhhhh_ Absolutely
3,2020-12-28 17:23:31,What should Tesla do with in-car gaming in an ...
4,2020-12-28 05:39:42,@PPathole @WSJ Absolutely
...,...,...
11712,2011-12-03 08:22:07,That was a total non sequitur btw
11713,2011-12-03 08:20:28,"Great Voltaire quote, arguably better than Twa..."
11714,2011-12-01 10:29:04,I made the volume on the Model S http://t.co/...
11715,2011-12-01 09:55:11,Went to Iceland on Sat to ride bumper cars on ...


In [5]:
tw['date']

0        2020-12-28 19:46:18
1        2020-12-28 18:07:33
2        2020-12-28 17:23:51
3        2020-12-28 17:23:31
4        2020-12-28 05:39:42
                ...         
11712    2011-12-03 08:22:07
11713    2011-12-03 08:20:28
11714    2011-12-01 10:29:04
11715    2011-12-01 09:55:11
11716    2010-06-04 18:31:57
Name: date, Length: 11717, dtype: object

In [6]:
test = tw['date'][0]
test

'2020-12-28 19:46:18'

In [7]:
re.findall('\d{4}-\d{2}-\d{2}', test)[0]

'2020-12-28'

In [8]:
def get_date(item):
    return re.findall('\d{4}-\d{2}-\d{2}', item)[0]

tw['date'] = tw['date'].apply(get_date)




# Tweet Frequency

In [9]:
tweet_freq = tw.groupby('date').agg({'date': 'count'})
tweet_freq

,date
date,
2010-06-04,1
2011-12-01,2
2011-12-03,2
2011-12-04,1
2011-12-21,7
...,...
2020-12-24,8
2020-12-25,12
2020-12-26,8


# Flags

## Tesla flag

In [10]:
re.findall('tesla', tw['tweet'][1000])

[]

In [11]:
def get_keys(tweet):
    if len(re.findall('T?t?esla', tweet)):
        return True
    else:
        return False

tw['tesla'] = tw['tweet'].apply(get_keys)

In [12]:
tw

,date,tweet,tesla
0,2020-12-28,Entertainment will be critical when cars drive...,False
1,2020-12-28,@kimpaquette Just meeting with Larry Ellison t...,True
2,2020-12-28,@richierichhhhh_ Absolutely,False
3,2020-12-28,What should Tesla do with in-car gaming in an ...,True
4,2020-12-28,@PPathole @WSJ Absolutely,False
...,...,...,...
11712,2011-12-03,That was a total non sequitur btw,False
11713,2011-12-03,"Great Voltaire quote, arguably better than Twa...",False
11714,2011-12-01,I made the volume on the Model S http://t.co/...,False
11715,2011-12-01,Went to Iceland on Sat to ride bumper cars on ...,False


In [13]:
tw['tesla'].value_counts() 

False    9450
True     2267
Name: tesla, dtype: int64

## Model Flags

In [14]:
def get_keys(tweet):
    if len(re.findall('M?m?odel', tweet)):
        return True
    else:
        return False


tw['model'] = tw['tweet'].apply(get_keys)
tw['model'].value_counts()

False    11266
True       451
Name: model, dtype: int64

## Sales flag 

In [15]:
def get_keys(tweet):
    if len(re.findall('s?S?ales', tweet)):
        return True
    else:
        return False


tw['sales'] = tw['tweet'].apply(get_keys)
tw['sales'].value_counts()

False    11682
True        35
Name: sales, dtype: int64

## Buy flag


In [16]:
def get_keys(tweet):
    if len(re.findall('B?b?uy', tweet)):
        return True
    else:
        return False


tw['buy'] = tw['tweet'].apply(get_keys)
tw['buy'].value_counts()

False    11471
True       246
Name: buy, dtype: int64

# Tesla Dataset

In [17]:
tsla = pd.read_csv('/Users/luismoreira/Downloads/TSLA.csv')
tsla

,Date,Open,High,Low,Close,Adj Close,Volume
0,2010-06-29,3.800000,5.000000,3.508000,4.778000,4.778000,93831500
1,2010-06-30,5.158000,6.084000,4.660000,4.766000,4.766000,85935500
2,2010-07-01,5.000000,5.184000,4.054000,4.392000,4.392000,41094000
3,2010-07-02,4.600000,4.620000,3.742000,3.840000,3.840000,25699000
4,2010-07-06,4.000000,4.000000,3.166000,3.222000,3.222000,34334500
...,...,...,...,...,...,...,...
2853,2021-10-27,1039.660034,1070.880005,1030.780029,1037.859985,1037.859985,38526500
2854,2021-10-28,1068.310059,1081.000000,1054.199951,1077.040039,1077.040039,27213200
2855,2021-10-29,1081.859985,1115.209961,1073.209961,1114.000000,1114.000000,29918400
2856,2021-11-01,1145.000000,1209.750000,1118.660034,1208.589966,1208.589966,56048700


In [18]:
tsla['Volatility'] = tsla['High'] - tsla['Low']
tsla['Day_diff'] = tsla['Close'] - tsla['Open']
tsla['Relative Volatility'] = (tsla['High'] - tsla['Low'])/tsla['High']*100
tsla.drop(['Adj Close'], axis = 1)

,Date,Open,High,Low,Close,Volume,Volatility,Day_diff,Relative Volatility
0,2010-06-29,3.800000,5.000000,3.508000,4.778000,93831500,1.492000,0.978000,29.840000
1,2010-06-30,5.158000,6.084000,4.660000,4.766000,85935500,1.424000,-0.392000,23.405654
2,2010-07-01,5.000000,5.184000,4.054000,4.392000,41094000,1.130000,-0.608000,21.797840
3,2010-07-02,4.600000,4.620000,3.742000,3.840000,25699000,0.878000,-0.760000,19.004329
4,2010-07-06,4.000000,4.000000,3.166000,3.222000,34334500,0.834000,-0.778000,20.850000
...,...,...,...,...,...,...,...,...,...
2853,2021-10-27,1039.660034,1070.880005,1030.780029,1037.859985,38526500,40.099976,-1.800049,3.744582
2854,2021-10-28,1068.310059,1081.000000,1054.199951,1077.040039,27213200,26.800049,8.729980,2.479190
2855,2021-10-29,1081.859985,1115.209961,1073.209961,1114.000000,29918400,42.000000,32.140015,3.766107
2856,2021-11-01,1145.000000,1209.750000,1118.660034,1208.589966,56048700,91.089966,63.589966,7.529652


In [19]:
tsla

,Date,Open,High,Low,Close,Adj Close,Volume,Volatility,Day_diff,Relative Volatility
0,2010-06-29,3.800000,5.000000,3.508000,4.778000,4.778000,93831500,1.492000,0.978000,29.840000
1,2010-06-30,5.158000,6.084000,4.660000,4.766000,4.766000,85935500,1.424000,-0.392000,23.405654
2,2010-07-01,5.000000,5.184000,4.054000,4.392000,4.392000,41094000,1.130000,-0.608000,21.797840
3,2010-07-02,4.600000,4.620000,3.742000,3.840000,3.840000,25699000,0.878000,-0.760000,19.004329
4,2010-07-06,4.000000,4.000000,3.166000,3.222000,3.222000,34334500,0.834000,-0.778000,20.850000
...,...,...,...,...,...,...,...,...,...,...
2853,2021-10-27,1039.660034,1070.880005,1030.780029,1037.859985,1037.859985,38526500,40.099976,-1.800049,3.744582
2854,2021-10-28,1068.310059,1081.000000,1054.199951,1077.040039,1077.040039,27213200,26.800049,8.729980,2.479190
2855,2021-10-29,1081.859985,1115.209961,1073.209961,1114.000000,1114.000000,29918400,42.000000,32.140015,3.766107
2856,2021-11-01,1145.000000,1209.750000,1118.660034,1208.589966,1208.589966,56048700,91.089966,63.589966,7.529652


In [27]:
tsla[tsla["Date"]=="2020-12-28"]

,Date,Open,High,Low,Close,Adj Close,Volume,Volatility,Day_diff,Relative Volatility
2643,2020-12-28,674.51001,681.400024,660.799988,663.690002,663.690002,32278600,20.600036,-10.820008,3.023193


In [28]:
tw[tw["date"]=="2020-12-28"]

,date,tweet,tesla,model,sales,buy
0,2020-12-28,Entertainment will be critical when cars drive...,False,False,False,False
1,2020-12-28,@kimpaquette Just meeting with Larry Ellison t...,True,False,False,False
2,2020-12-28,@richierichhhhh_ Absolutely,False,False,False,False
3,2020-12-28,What should Tesla do with in-car gaming in an ...,True,False,False,False
4,2020-12-28,@PPathole @WSJ Absolutely,False,False,False,False
5,2020-12-28,@Tesmanian_com Taste of the future,False,False,False,False


In [21]:
merged = tsla.merge(tw, left_on='Date', right_on='date', how='left')#.shape

In [39]:
merged = merged.fillna(False)

In [23]:
merged.columns

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume',
       'Volatility', 'Day_diff', 'Relative Volatility', 'date', 'tweet',
       'tesla', 'model', 'sales', 'buy'],
      dtype='object')

In [29]:
merged[merged['Date'] == '2010-07-01']

,Date,Open,High,Low,Close,Adj Close,Volume,Volatility,Day_diff,Relative Volatility,date,tweet,tesla,model,sales,buy
2,2010-07-01,5.0,5.184,4.054,4.392,4.392,41094000,1.13,-0.608,21.79784,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
merged.columns

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume',
       'Volatility', 'Day_diff', 'Relative Volatility', 'date', 'tweet',
       'tesla', 'model', 'sales', 'buy'],
      dtype='object')

In [41]:
merged[merged['Date'] == '2020-12-28']

,Date,Open,High,Low,Close,Adj Close,Volume,Volatility,Day_diff,Relative Volatility,date,tweet,tesla,model,sales,buy
9333,2020-12-28,674.51001,681.400024,660.799988,663.690002,663.690002,32278600,20.600036,-10.820008,3.023193,2020-12-28,Entertainment will be critical when cars drive...,False,False,False,False
9334,2020-12-28,674.51001,681.400024,660.799988,663.690002,663.690002,32278600,20.600036,-10.820008,3.023193,2020-12-28,@kimpaquette Just meeting with Larry Ellison t...,True,False,False,False
9335,2020-12-28,674.51001,681.400024,660.799988,663.690002,663.690002,32278600,20.600036,-10.820008,3.023193,2020-12-28,@richierichhhhh_ Absolutely,False,False,False,False
9336,2020-12-28,674.51001,681.400024,660.799988,663.690002,663.690002,32278600,20.600036,-10.820008,3.023193,2020-12-28,What should Tesla do with in-car gaming in an ...,True,False,False,False
9337,2020-12-28,674.51001,681.400024,660.799988,663.690002,663.690002,32278600,20.600036,-10.820008,3.023193,2020-12-28,@PPathole @WSJ Absolutely,False,False,False,False
9338,2020-12-28,674.51001,681.400024,660.799988,663.690002,663.690002,32278600,20.600036,-10.820008,3.023193,2020-12-28,@Tesmanian_com Taste of the future,False,False,False,False
